In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
session = requests.Session()
url = "https://sportsbook.draftkings.com/leagues/football/2?category=game-lines&subcategory=game"
r = session.get(url)

In [5]:
soup = BeautifulSoup(r.text, "html.parser")

In [10]:
outer = soup.find_all("div", {"class": "parlay-card-10-a"})

In [11]:
all_data = {}

In [20]:
for o in outer:
    i = 0
    
    tags = o.find_all("div", {"class": "sportsbook-table__body"})[0].children
    
    for tag in tags:
        
        if i not in all_data:
            all_data[i] = []
            
        if i == 0:
            rows = tag.find_all("span", {"class": "event-cell__name"})
            for row in rows:
                all_data[0].append(row.text)
                
        elif i == 1:
            rows = tag.find_all("div", {"class": "sportsbook-table__column-row"})
            for row in rows:
                if len(row.find_all("div", {"class": "sportsbook-empty-cell body"})) == 0:

                    line = row.find_all("span", {"class": "sportsbook-outcome-cell__line"})[0].text
                    odds = row.find_all("span", {"class": "sportsbook-odds"})[0].text
                    all_data[1].append((line, odds))
                else:
                    all_data[1].append(None)
                    
                    
        elif i == 2:

            rows = tag.find_all("div", {"class": "sportsbook-table__column-row"})
            for j in range(1, len(rows), 2):
                if len(rows[j].find_all("div", {"class": "sportsbook-empty-cell body"})) == 0:


                    over = rows[j].find_all("div", {"class": "sportsbook-outcome-cell__body"})[0]
                    under = rows[j-1].find_all("div", {"class": "sportsbook-outcome-cell__body"})[0]


                    over_tuple = (over['aria-label'], over.find_all("span", {"class": "sportsbook-odds"})[0].text)
                    under_tuple = (under['aria-label'], under.find_all("span", {"class": "sportsbook-odds"})[0].text)

                    all_data[2].append((over_tuple, under_tuple))
                    all_data[2].append((over_tuple, under_tuple))

                else:
                    all_data[2].append(None)
                    all_data[2].append(None)
                
        elif i == 3:

            rows = tag.find_all("div", {"class": "sportsbook-table__column-row"})
            for row in rows:
                if len(row.find_all("div", {"class": "sportsbook-empty-cell body"})) == 0:

                    ml = row.find_all("span", {"class": "sportsbook-odds"})[0].text


                    all_data[3].append(ml)
                else:
                    all_data[3].append(None)
                    
        
        i += 1

In [22]:
pd.DataFrame(all_data)

ValueError: arrays must all be same length